In [25]:
# Import libraries
import os
import openai
import pandas as pd
import re
from dotenv import load_dotenv

# Load OpenAI API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OpenAI API Key loaded successfully!")
else:
    print("OpenAI API Key not found.")
client = openai.OpenAI(api_key=openai_api_key)

# Define the five metacognitive prompting (MP) dimensions
MP_DIMENSIONS = ['Comprehension', 'Judgment', 'Evaluation', 'Final Decision', 'Confidence']


# Function to extract continuous score from GPT reply
def extract_probabilistic_scores(reply):
    parsed = {}
    for dim in MP_DIMENSIONS:
        key = dim.lower().replace(' ', '_')
        match = re.search(rf"{dim}:\s*\[([^\]]+)\]", reply, re.IGNORECASE)
        if match:
            try:
                raw_probs = match.group(1).split(",")
                probs = [float(p.strip()) for p in raw_probs]
                if len(probs) == 5 and abs(sum(probs) - 1.0) < 0.1:
                    expected = sum(p * i for i, p in enumerate(probs))
                    parsed[f"{key}_continuous"] = round(expected, 2)
                else:
                    parsed[f"{key}_continuous"] = None
            except:
                parsed[f"{key}_continuous"] = None
        else:
            parsed[f"{key}_continuous"] = None
    return parsed


# GPT-4 based MP scoring function
def gpt_mp_score_continuous_combined(df_subset, model="gpt-4-turbo"):
    results = []

    for idx, row in df_subset.iterrows():
        meta = row.to_dict()
        text = str(meta["combined_reflection"]).strip()

        if not text or len(text) < 10:
            auto_filled = {f"{dim.lower().replace(' ', '_')}_continuous": 0.0 for dim in MP_DIMENSIONS}
            results.append({**meta, **auto_filled, "gpt_rationale": "Auto-scored due to empty or trivial reflection."})
            continue

        try:
            prompt = f"""
You are a metacognitive scientist evaluating a participant's reflection on a three-choice probabilistic reversal learning task.

The following narrative is a response to two reflection questions:
1. What strategy did you use?
2. Did you switch strategies during the task?

Your goal is to evaluate this reflection using the Metacognitive Prompting (MP) framework.

---

**MP Dimensions (Wang & Zhao, 2025):**

You will score the reflection across the following five dimensions:

1. **Comprehension** – Did the participant understand the task rules, structure, or goal?  
2. **Judgment** – Did they form a hypothesis or plan about how to act?  
3. **Evaluation** – Did they reflect on whether their strategy worked, and why?  
4. **Final Decision** – Did they describe a consistent rule or decision strategy they ultimately applied?  
5. **Confidence** – Did they express certainty or doubt about their approach or understanding?

---

**Scoring Rubric (used for each dimension):**  
Each dimension should be scored on a scale from 0 to 4, where:

- **0 = Absent** – No evidence of the dimension  
- **1 = Rudimentary** – Vague or minimal expression  
- **2 = Adequate** – Some clarity but not in-depth  
- **3 = Good** – Clear and thoughtful expression  
- **4 = Exemplary** – Sophisticated and comprehensive insight

---

**Scoring Guidance:**  
Please recognize that high-quality metacognition may appear in many forms. Participants may demonstrate deep insight even if they do not explicitly state phrases like “I evaluated” or “I was confident.” You should reward:

- Conditional reasoning (e.g., "if... then..." statements)  
- Hypothetical thinking or counterfactuals  
- Thoughtful restraint and uncertainty  
- Philosophical or cautious language (e.g., “presumption of innocence”)  
- Rational inertia or minimal strategy change, when justified  
- Reflective self-awareness (of effort, bias, or internal limits)

Short responses can still be insightful. Do not penalize indirect or abstract language if the logic and self-awareness are present.

---

**Your Task:**  
For each dimension, estimate the probability that the reflection deserves a score of 0, 1, 2, 3, or 4.  
Respond using decimal values only (e.g., `[0.1, 0.2, 0.3, 0.2, 0.2]`).  
The values must:

- Be enclosed in square brackets  
- Contain exactly 5 decimals  
- Sum to 1.0 (±0.01)  
- Be purely numeric (no letters)

Format your output like this:

Comprehension: [0.0, 0.1, 0.2, 0.3, 0.4]  
Judgment: [0.2, 0.2, 0.2, 0.2, 0.2]  
Evaluation: [0.9, 0.1, 0.0, 0.0, 0.0]  
Final Decision: [0.0, 0.0, 0.0, 0.2, 0.8]  
Confidence: [0.1, 0.1, 0.1, 0.3, 0.4]  

Rationale: Provide a brief explanation for your probability estimates.

**Scoring Tips:**
- If the reflection shows deep reasoning, nuance, self-awareness, or philosophical restraint, it is okay to assign something like:  
  `[0.0, 0.0, 0.0, 0.2, 0.8]`. So, you give this response a 80% probability of scoring closer to a 4 (an exemplary rating) for a particular dimension.  
- If the dimension is completely absent, something like:  
  `[0.9, 0.1, 0.0, 0.0, 0.0]` is acceptable. So, you give this response a 90% probability of scoring closer to a 0 (an absent rating) for a particular dimension.  
- Use the full scale honestly and appropriately. Reflections with strong insight deserve strong scores (> 0.7).

---

**Reflection:**  
\"\"\"{text}\"\"\"
"""

            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0
            )

            reply = response.choices[0].message.content.strip()
            parsed_scores = extract_probabilistic_scores(reply)

            results.append({
                **meta,
                **parsed_scores,
                "gpt_rationale": reply
            })

        except Exception as e:
            print(f"Error on row {idx}: {e}")
            error_filled = {f"{dim.lower().replace(' ', '_')}_continuous": None for dim in MP_DIMENSIONS}
            results.append({**meta, **error_filled, "gpt_rationale": "error"})

    return pd.DataFrame(results)


# Load data
df = pd.read_csv("hmp_paranoia.csv")

strategy_col = "Did you use any particular strategy or strategies? If yes, please describe."
switching_col = "Did you find yourself switching strategies over the course of the game?"

df['strategy'] = df[strategy_col].fillna("").astype(str).str.strip()
df['switching'] = df[switching_col].fillna("").astype(str).str.strip()
df['combined_reflection'] = df['strategy'] + " ; " + df['switching']

# Keep only relevant columns
cols_to_keep = [
    'id', 'mturk_id', 'paranoia_score', 'referential_group', 'persecution_group',
    'Did any of the partners deliberately sabotage you?',
    'wsr_avg', 'mu02_avg', 'mu03_avg', 'kappa_avg', 'omega2_avg', 'omega3_avg',
    'strategy', 'switching', 'combined_reflection'
]
df_subset = df[cols_to_keep].copy()

# Run GPT-MP scoring function
df_final = gpt_mp_score_continuous_combined(df_subset)

# Reorder final columns
final_cols = cols_to_keep + [f"{dim.lower().replace(' ', '_')}_continuous" for dim in MP_DIMENSIONS] + ['gpt_rationale']
df_final = df_final[[col for col in final_cols if col in df_final.columns]]


OpenAI API Key loaded successfully!


In [26]:
print(df_final.head())

   id        mturk_id  paranoia_score referential_group persecution_group  \
0   1  A2541C8MY0BYV3        0.000000               low               low   
1   2  A3JSDZMBS8L87S        0.722222               low               low   
2   3  A1PR74OHURJNTO        0.250000               low               low   
3   4   A1CSDIX05PK9V        0.000000               low               low   
4   5  A1JJYY622DGE5L        0.444444               low               low   

   Did any of the partners deliberately sabotage you?   wsr_avg  mu02_avg  \
0                                               -2.0   0.000000 -0.134659   
1                                               -2.0   0.017857 -0.326276   
2                                               -3.0   0.000000 -0.011850   
3                                               -2.0   0.010000 -0.129578   
4                                               -2.0   0.000000 -0.914451   

   mu03_avg  kappa_avg  ...  omega3_avg  \
0 -0.035434   0.428227  ...   -

In [27]:
# Save to CSV
df_final.to_csv("mp_scores_continuous_500.csv", index=False)